In [1]:
import spacy
from zss import Node, simple_distance
from functools import partial
# Imports
import numpy as np
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
tqdm.pandas(desc='Processing text')
# Load the French language model
nlp = spacy.load("fr_core_news_md")

In [2]:
import spacy
from zss import Node, simple_distance
from functools import partial

# Function to convert a spaCy token to a zss Node recursively
def token_to_zss_node(mode, token):
    if mode=="dep":
        node = Node(token.dep_)
    elif mode=="text":
        node = Node(token.orth_)
    elif mode=="pos":
        node = Node(token.pos_)
        
    for child in token.children:
        node.addkid(token_to_zss_node(mode=mode, token=child))
    return node

def build_zss_tree(spacy_doc, zss_parser):
    root = Node("ROOT")
    for sent in spacy_doc.sents:
        root.addkid(zss_parser(sent.root))
    return root

# Function to compute tree edit distance between two documents
def compute_tree_edit_distance(parsed_doc1, parsed_doc2, zss_parser):
    root1 = build_zss_tree(spacy_doc=parsed_doc1,zss_parser=zss_parser)
    root2 = build_zss_tree(spacy_doc=parsed_doc2,zss_parser=zss_parser)
    # Compute the tree edit distance using zss
    return simple_distance(root1, root2)

def _remove_stop_words(doc):
    return " ".join([str(t) for t in doc if not t.is_stop])

def semantic_similarity(doc1, doc2):
    return doc1.similarity(doc2)

# zss parser
zss_dependency_parser= partial(token_to_zss_node,'dep')

# Construct Questions Entries

In [3]:
conn = duckdb.connect(database="test.db", read_only=False)
cur = conn.cursor()
cur.execute("CREATE OR REPLACE TABLE questions AS SELECT * FROM './data/questions_fr_train.csv';")

In [17]:
cur.execute("CREATE OR REPLACE TABLE questions_simple AS SELECT * FROM './data/questions_syn_simple_ask.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_simple_tc AS SELECT * FROM './data/questions_syn_simple_ask_qcc.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_simple_tc_fewshots AS SELECT * FROM './data/questions_syn_simple_ask_qcc_fewshots.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe AS SELECT * FROM './data/questions_syn_describe_then_ask.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe_tc AS SELECT * FROM './data/questions_syn_describe_then_ask_qcc.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe_fewshots AS SELECT * FROM './data/questions_syn_describe_then_ask_fewshots.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe_tc_fewshots AS SELECT * FROM './data/questions_syn_describe_then_ask_qcc_fewshots.csv';")

In [75]:
cur.execute("CREATE OR REPLACE TABLE questions_syn_extrapolation AS SELECT * FROM './data/questions_syn_extrapolation.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_syn_extrapolation_10_seed_data AS SELECT * FROM './data/questions_syn_extrapolation_10_seed_data.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_syn_extrapolation_50_seed_data_round_0 AS SELECT * FROM './data/questions_syn_extrapolation_50_seed_data_round_0.csv';")

In [76]:
dfQ = cur.execute("""
SELECT questions.article_ids, question
FROM questions
""").df()

dfQ['article_ids'] = dfQ['article_ids'].str.split(',')
dfQ = dfQ.explode('article_ids')

cur.execute("""
CREATE OR REPLACE TABLE questions AS 
SELECT * FROM dfQ
""")

cur.execute("""
SELECT * FROM questions
""").df()

,article_ids,question
0,22225,Je suis travailleur salarié(e). Puis-je refuse...
1,22226,Je suis travailleur salarié(e). Puis-je refuse...
2,22227,Je suis travailleur salarié(e). Puis-je refuse...
3,22228,Je suis travailleur salarié(e). Puis-je refuse...
4,22229,Je suis travailleur salarié(e). Puis-je refuse...
...,...,...
5779,14479,Si je ne rembourse pas mon crédit hypothécaire...
5780,14480,Si je ne rembourse pas mon crédit hypothécaire...
5781,14481,Si je ne rembourse pas mon crédit hypothécaire...
5782,14482,Si je ne rembourse pas mon crédit hypothécaire...


In [78]:
cur.execute("""
    CREATE OR REPLACE TABLE questions_summary as 
    (SELECT questions.article_ids, questions.question,
    questions_simple.synthetic_question as questions_simple, 
    questions_simple_tc.synthetic_question as questions_simple_tc, 
    questions_simple_tc_fewshots.synthetic_question as questions_simple_tc_fewshots,
    questions_describe.synthetic_question as questions_describe, 
    questions_describe_tc.synthetic_question as questions_describe_tc, 
    questions_describe_fewshots.synthetic_question as questions_describe_fewshots, 
    questions_describe_tc_fewshots.synthetic_question as questions_describe_tc_fewshots,
    questions_syn_extrapolation.synthetic_question as questions_syn_extrapolation,
    questions_syn_extrapolation_10_seed_data.synthetic_question as questions_syn_extrapolation_10_seed_data,
    questions_syn_extrapolation_50_seed_data_round_0.synthetic_question as questions_syn_extrapolation_50_seed_data_round_0
    FROM questions, questions_simple, questions_simple_tc, questions_simple_tc_fewshots, questions_describe, questions_describe_tc, questions_describe_fewshots, questions_describe_tc_fewshots, questions_syn_extrapolation,questions_syn_extrapolation_10_seed_data, questions_syn_extrapolation_50_seed_data_round_0
    where questions.article_ids not like '%,%'
    and questions.article_ids = questions_simple.article_ids
    and questions.article_ids = questions_simple_tc.article_ids
    and questions.article_ids = questions_simple_tc_fewshots.article_ids
    and questions.article_ids = questions_describe.article_ids
    and questions.article_ids = questions_describe_tc.article_ids
    and questions.article_ids = questions_describe_fewshots.article_ids
    and questions.article_ids = questions_describe_tc_fewshots.article_ids
    and questions.article_ids = questions_syn_extrapolation.article_ids
    and questions.article_ids = questions_syn_extrapolation_10_seed_data.article_ids
    and questions.article_ids = questions_syn_extrapolation_50_seed_data_round_0.article_ids
    )
""")

In [79]:
dfQ_summary = cur.execute("""
    SELECT *
    FROM questions_summary
""").df()
dfQ_summary

,article_ids,question,questions_simple,questions_simple_tc,questions_simple_tc_fewshots,questions_describe,questions_describe_tc,questions_describe_fewshots,questions_describe_tc_fewshots,questions_syn_extrapolation,questions_syn_extrapolation_10_seed_data,questions_syn_extrapolation_50_seed_data_round_0
0,1115,Les parents ne sont pas mariés ensemble. La mè...,Quels sont les modes de preuve de la filiation...,Quel est le critère principal pour prouver la ...,La coparente prétendue peut-elle prouver sa fi...,Une personne revendique la filiation à l'égard...,Un couple a recours à un service de médiation ...,Une personne prétend être le parent d'un enfan...,La filiation d'un enfant est contestée par une...,La filiation d'un enfant est contestée par une...,La filiation d'un enfant est contestée par une...,La filiation d'un enfant est contestée par une...
1,2535,Je loue un garage. Quelles règles s'appliquent...,Quelles conséquences juridiques entraîne l'ins...,Quel est l'effet de la clause résolutoire expr...,La résolution d’un contrat commercial peut-ell...,Un locataire a signé un bail commercial avec u...,Un consommateur a engagé une procédure de règl...,Vous avez conclu un contrat comportant une cla...,Deux parties ont inclus une clause résolutoire...,Deux parties ont inclus une clause résolutoire...,Deux parties ont inclus une clause résolutoire...,Deux parties ont inclus une clause résolutoire...
2,4462,Quels recours exercer contre les abus lors d'u...,Quel est l'organe chargé de représenter l'ordr...,Est-ce que le Conseil d'administration représe...,Est-ce que le Conseil d'administration représe...,Le Conseil d'administration d'un ordre profess...,Un consommateur a un litige avec une entrepris...,Un avocat agit au nom de son ordre professionn...,Un avocat souhaite engager des poursuites judi...,Un avocat souhaite engager des poursuites judi...,Un avocat souhaite engager des poursuites judi...,Un avocat souhaite engager des poursuites judi...
3,4557,Quel est le coût de l'huissier de justice ?,Quelles sont les fonctions spécifiques des mem...,"La question serait : ""Est-il obligatoire qu'au...",Les membres du conseil de la chambre d'arrondi...,Il s'agit des dispositions légales relatives à...,Dans le cadre du règlement des litiges de cons...,Le conseil de la chambre d'arrondissement des ...,Dans une chambre d'arrondissement des huissier...,Dans une chambre d'arrondissement des huissier...,Dans une chambre d'arrondissement des huissier...,Dans une chambre d'arrondissement des huissier...
4,1093,Quelles démarches effectuer après l'accoucheme...,Quelles personnes peuvent contester la filiati...,Est-ce que la contestation de la filiation mat...,Peut-on contester la filiation maternelle étab...,Un homme découvre que la filiation maternelle ...,Une femme découvre que la filiation maternelle...,Une personne pense que la filiation maternelle...,Une personne prétend que la filiation maternel...,Une personne prétend que la filiation maternel...,Je suis le père de l'enfant et la mère de l'en...,Une personne prétend que la filiation maternel...
...,...,...,...,...,...,...,...,...,...,...,...,...
37144,5955,Qu'est-ce que le règlement collectif de dettes...,Quelles informations doivent être incluses dan...,Quelles sont les mentions obligatoires qui doi...,Voulez-vous introduire une demande de règlemen...,Un particulier a accumulé des dettes et souhai...,Un individu souhaite introduire une demande de...,Je cherche à déposer une demande de règlement ...,Je souhaite introduire une demande de règlemen...,Je fais une demande de règlement collectif de ...,Je rencontre des difficultés financières et je...,Je souhaite introduire une demande de règlemen...
37145,5955,Comment demander un règlement collectif de det...,Quelles informations doivent être incluses dan...,Quelles sont les mentions obligatoires qui doi...,Voulez-vous introduire une demande de règlemen...,Un particulier a accumulé des dettes et souhai...,Un individu souhaite introduire une de

In [80]:
dfQ_summary = dfQ_summary.sample(n=2000,random_state=42)

In [133]:
dfQ_summary['simple_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_simple']),zss_parser=zss_dependency_parser), axis=1)

In [134]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_simple_tc_fewshots']),zss_parser=zss_dependency_parser), axis=1)

dfQ_summary['simple_tc_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_simple_tc']),zss_parser=zss_dependency_parser), axis=1)

In [135]:
dfQ_summary['describe_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe']),zss_parser=zss_dependency_parser), axis=1)

dfQ_summary['describe_tc_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe_tc']),zss_parser=zss_dependency_parser), axis=1)

In [136]:
dfQ_summary['describe_fewshots_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe_fewshots']),zss_parser=zss_dependency_parser), axis=1)

dfQ_summary['describe_tc_fewshots_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe_tc_fewshots']),zss_parser=zss_dependency_parser), axis=1)

In [17]:
dfQ_summary['questions_syn_extrapolation_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_syn_extrapolation']),zss_parser=zss_dependency_parser), axis=1)

In [42]:
dfQ_summary['questions_syn_extrapolation_10_seed_data_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_syn_extrapolation_10_seed_data']),zss_parser=zss_dependency_parser), axis=1)

In [81]:
dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_syn_extrapolation_50_seed_data_round_0']),zss_parser=zss_dependency_parser), axis=1)

In [82]:
dfQ_summary

,article_ids,question,questions_simple,questions_simple_tc,questions_simple_tc_fewshots,questions_describe,questions_describe_tc,questions_describe_fewshots,questions_describe_tc_fewshots,questions_syn_extrapolation,questions_syn_extrapolation_10_seed_data,questions_syn_extrapolation_50_seed_data_round_0,questions_syn_extrapolation_50_seed_data_round_0_tree_edit_distance
70,858,Le propriétaire peut-il mettre fin au bail non...,Quelles sont les conditions permettant au pren...,Est-ce que le preneur d'un contrat de bail de ...,"Peut un locataire d'un bail de courte durée, é...",En raison de la crise sanitaire liée au Covid-...,Un étudiant locataire sous bail de courte duré...,"En raison de la crise sanitaire du Covid-19, u...",En raison de la crise sanitaire causée par le ...,En raison de la crise sanitaire causée par le ...,Je loue un appartement à Bruxelles depuis janv...,En raison de la crise sanitaire causée par le ...,66.0
12503,1098,"Nous sommes mariés. Nous sommes séparés, vais-...",Est-ce que la présomption de paternité prévue ...,Est-ce que la présomption de paternité visée à...,Les parents ont fait une déclaration conjointe...,Deux époux ont décidé de résider séparément su...,Deux adultes habitant ensemble depuis plusieur...,Un couple n'a pas fait de déclaration conjoint...,Les époux ont vécu séparément un certain temps...,Les époux ont vécu séparément un certain temps...,Un enfant est né lorsque les parents étaient s...,Les époux ont vécu séparément un certain temps...,32.0
35580,4702,Comment savoir quel est le tribunal de la fami...,Quel tribunal est compétent pour traiter des d...,La résidence habituelle d'un enfant est-elle d...,Si des parents séparés ont des enfants communs...,Deux anciens cohabitants légaux souhaitent rég...,Un couple marié envisage de faire appel au tri...,Les parents sont en conflit concernant l'héber...,Une personne souhaite entamer une procédure ju...,Une personne souhaite entamer une procédure ju...,Je souhaite entamer une action en matière de f...,Une personne souhaite entamer une procédure ju...,32.0
24112,881,Peut-on réclamer un loyer à un colocataire qui...,Quelles sont les règles applicables en cas de ...,Est-ce que le bailleur peut mettre fin au bail...,Est-ce que tous les colocataires doivent signe...,Deux colocataires partageant un appartement dé...,Un colocataire souhaite sortir anticipativemen...,Je vis en colocation et je souhaite quitter l'...,Je vis en colocation avec d'autres personnes. ...,Je souhaite quitter ma colocation en Belgique....,Une personne désire partir en cours de bail en...,Je vis en colocation avec d'autres personnes. ...,33.0
25030,881,Un colocataire seul peut-il mettre fin au bail...,Quelles sont les règles applicables en cas de ...,Est-ce que le bailleur peut mettre fin au bail...,Est-ce que tous les colocataires doivent signe...,Deux colocataires partageant un appartement dé...,Un colocataire souhaite sortir anticipativemen...,Je vis en colocation et je souhaite quitter l'...,Je vis en colocation avec d'autres personnes. ...,Trois colocataires ont signé un bail de coloca...,Je souhaite sortir anticipativement d'un bail ...,Je vis en colocation avec d'autres personnes. ...,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,2334,"Je me sépare. On se sépare, qui doit payer le ...",Quelles sont les obligations financières des c...,Est-ce que les cohabitants légaux sont tenus d...,Est-ce que les cohabitants légaux sont tenus d...,Deux personnes vivent en tant que cohabitants ...,Deux cohabitants légaux partagent une vie comm...,Deux personnes vivent ensemble en tant que coh...,Deux personnes vivent en cohabitation légale e...,Deux cohabitants légaux se séparent et ont des...,Deux cohabitants légaux se séparent après avoi...,Deux personnes vivent en cohabitation légale e...,39.0
30001,2334,Mon cohabitant légal est décédé. Quels sont me...,Quelles sont les obligations financières des c...,Est-ce que les cohabitants légaux sont tenus d...,Est-ce que 

In [138]:
dfQ_summary['simple_tree_edit_distance'].mean()

22.7875

In [139]:
dfQ_summary['simple_tc_tree_edit_distance'].mean()

23.953

In [140]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance'].mean()

25.211

In [141]:
dfQ_summary['describe_tree_edit_distance'].mean()

55.5865

In [142]:
dfQ_summary['describe_tc_tree_edit_distance'].mean()

50.4745

In [143]:
dfQ_summary['describe_fewshots_tree_edit_distance'].mean()

32.899

In [144]:
dfQ_summary['describe_tc_fewshots_tree_edit_distance'].mean()

32.9725

In [19]:
dfQ_summary['questions_syn_extrapolation_tree_edit_distance'].mean()

33.3115

In [88]:
dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_tree_edit_distance'].mean()

35.9235

# semantic 

In [86]:
dfQ_summary['simple_tc_fewshots_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_simple_tc_fewshots'])))), axis=1)

dfQ_summary['simple_tc_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_simple_tc'])))), axis=1)

In [87]:
dfQ_summary['simple_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_simple'])))), axis=1)

In [70]:
dfQ_summary['describe_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe'])))), axis=1)

dfQ_summary['describe_tc_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe_tc'])))), axis=1)

dfQ_summary['describe_fewshots_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe_fewshots'])))), axis=1)

dfQ_summary['describe_tc_fewshots_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe_tc_fewshots'])))), axis=1)

In [25]:
dfQ_summary['questions_syn_extrapolation_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_syn_extrapolation'])))), axis=1)

In [96]:
dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_syn_extrapolation_50_seed_data_round_0'])))), axis=1)

In [71]:
dfQ_summary['describe_semantic_similarity'].mean()

0.5535614752042263

In [72]:
dfQ_summary['describe_tc_semantic_similarity'].mean()

0.5656643756623633

In [73]:
dfQ_summary['describe_fewshots_semantic_similarity'].mean()

0.603698511885458

In [74]:
dfQ_summary['describe_tc_fewshots_semantic_similarity'].mean()

0.6085111030132824

In [81]:
dfQ_summary['simple_tc_fewshots_semantic_similarity'].mean()

0.5915320833611377

In [89]:
dfQ_summary['simple_semantic_similarity'].mean()

0.5464909524748509

In [90]:
dfQ_summary['simple_tc_semantic_similarity'].mean()

0.576864490860546

In [93]:
dfQ_summary['simple_tc_fewshots_semantic_similarity'].mean()

0.5915320833611377

In [26]:
dfQ_summary['questions_syn_extrapolation_semantic_similarity'].mean()

0.6528104574938352

In [97]:
dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_semantic_similarity'].mean()

0.6874022130523638

# sentence length difference normalization

In [145]:
dfQ_summary['simple_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_simple'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [146]:
dfQ_summary['simple_tc_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_simple_tc'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [147]:
dfQ_summary['simple_tc_fewshots_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_simple_tc_fewshots'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [148]:
dfQ_summary['describe_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [149]:
dfQ_summary['describe_fewshots_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe_fewshots'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [150]:
dfQ_summary['describe_tc_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe_tc'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [151]:
dfQ_summary['describe_tc_fewshots_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe_tc_fewshots'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [20]:
dfQ_summary['questions_syn_extrapolation_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_syn_extrapolation'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [48]:
dfQ_summary['questions_syn_extrapolation_10_seed_data_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_syn_extrapolation_10_seed_data'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [89]:
dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_syn_extrapolation_50_seed_data_round_0'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [85]:
dfQ_summary

,article_ids,question,questions_simple,questions_simple_tc,questions_simple_tc_fewshots,questions_describe,questions_describe_tc,questions_describe_fewshots,questions_describe_tc_fewshots,questions_syn_extrapolation,questions_syn_extrapolation_10_seed_data,questions_syn_extrapolation_50_seed_data_round_0,questions_syn_extrapolation_50_seed_data_round_0_tree_edit_distance,questions_syn_extrapolation_50_seed_data_round_0_length_diff
70,858,Le propriétaire peut-il mettre fin au bail non...,Quelles sont les conditions permettant au pren...,Est-ce que le preneur d'un contrat de bail de ...,"Peut un locataire d'un bail de courte durée, é...",En raison de la crise sanitaire liée au Covid-...,Un étudiant locataire sous bail de courte duré...,"En raison de la crise sanitaire du Covid-19, u...",En raison de la crise sanitaire causée par le ...,En raison de la crise sanitaire causée par le ...,Je loue un appartement à Bruxelles depuis janv...,En raison de la crise sanitaire causée par le ...,66.0,51
12503,1098,"Nous sommes mariés. Nous sommes séparés, vais-...",Est-ce que la présomption de paternité prévue ...,Est-ce que la présomption de paternité visée à...,Les parents ont fait une déclaration conjointe...,Deux époux ont décidé de résider séparément su...,Deux adultes habitant ensemble depuis plusieur...,Un couple n'a pas fait de déclaration conjoint...,Les époux ont vécu séparément un certain temps...,Les époux ont vécu séparément un certain temps...,Un enfant est né lorsque les parents étaient s...,Les époux ont vécu séparément un certain temps...,32.0,21
35580,4702,Comment savoir quel est le tribunal de la fami...,Quel tribunal est compétent pour traiter des d...,La résidence habituelle d'un enfant est-elle d...,Si des parents séparés ont des enfants communs...,Deux anciens cohabitants légaux souhaitent rég...,Un couple marié envisage de faire appel au tri...,Les parents sont en conflit concernant l'héber...,Une personne souhaite entamer une procédure ju...,Une personne souhaite entamer une procédure ju...,Je souhaite entamer une action en matière de f...,Une personne souhaite entamer une procédure ju...,32.0,20
24112,881,Peut-on réclamer un loyer à un colocataire qui...,Quelles sont les règles applicables en cas de ...,Est-ce que le bailleur peut mettre fin au bail...,Est-ce que tous les colocataires doivent signe...,Deux colocataires partageant un appartement dé...,Un colocataire souhaite sortir anticipativemen...,Je vis en colocation et je souhaite quitter l'...,Je vis en colocation avec d'autres personnes. ...,Je souhaite quitter ma colocation en Belgique....,Une personne désire partir en cours de bail en...,Je vis en colocation avec d'autres personnes. ...,33.0,18
25030,881,Un colocataire seul peut-il mettre fin au bail...,Quelles sont les règles applicables en cas de ...,Est-ce que le bailleur peut mettre fin au bail...,Est-ce que tous les colocataires doivent signe...,Deux colocataires partageant un appartement dé...,Un colocataire souhaite sortir anticipativemen...,Je vis en colocation et je souhaite quitter l'...,Je vis en colocation avec d'autres personnes. ...,Trois colocataires ont signé un bail de coloca...,Je souhaite sortir anticipativement d'un bail ...,Je vis en colocation avec d'autres personnes. ...,33.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,2334,"Je me sépare. On se sépare, qui doit payer le ...",Quelles sont les obligations financières des c...,Est-ce que les cohabitants légaux sont tenus d...,Est-ce que les cohabitants légaux sont tenus d...,Deux personnes vivent en tant que cohabitants ...,Deux cohabitants légaux partagent une vie comm...,Deux personnes vivent ensemble en tant que coh...,Deux personnes vivent en cohabitation légale e...,Deux cohabitants légaux se séparent et ont des...,Deux cohabitants légaux se séparent après avoi...,Deux personnes vivent en cohabitation légale e...,39.0,28
30001,2334,Mon cohabitant légal est décédé. Quels sont me...,Quelles sont les obligations

In [153]:
dfQ_summary['simple_tree_edit_distance_norm'] = dfQ_summary['simple_tree_edit_distance'] - dfQ_summary['simple_length_diff']

In [154]:
dfQ_summary['simple_tc_tree_edit_distance_norm'] = dfQ_summary['simple_tc_tree_edit_distance'] - dfQ_summary['simple_tc_length_diff']

In [155]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance_norm'] = dfQ_summary['simple_tc_fewshots_tree_edit_distance'] - dfQ_summary['simple_tc_fewshots_length_diff']

In [156]:
dfQ_summary['describe_tree_edit_distance_norm'] = dfQ_summary['describe_tree_edit_distance'] - dfQ_summary[
    'describe_length_diff']

In [157]:
dfQ_summary['describe_fewshots_tree_edit_distance_norm'] = dfQ_summary['describe_fewshots_tree_edit_distance'] - dfQ_summary['describe_fewshots_length_diff']

In [158]:
dfQ_summary['describe_tc_tree_edit_distance_norm'] = dfQ_summary['describe_tc_tree_edit_distance'] - dfQ_summary['describe_tc_length_diff']

In [159]:
dfQ_summary['describe_tc_fewshots_tree_edit_distance_norm'] = dfQ_summary['describe_tc_fewshots_tree_edit_distance'] - dfQ_summary['describe_tc_fewshots_length_diff']

In [23]:
dfQ_summary['questions_syn_extrapolation_tree_edit_distance_norm'] = dfQ_summary['questions_syn_extrapolation_tree_edit_distance'] - dfQ_summary['questions_syn_extrapolation_length_diff']

In [91]:
dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_tree_edit_distance_norm'] = dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_tree_edit_distance'] - dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_length_diff']

In [160]:
dfQ_summary['simple_tree_edit_distance_norm'].mean()

12.938

In [161]:
dfQ_summary['simple_tc_tree_edit_distance_norm'].mean()

13.8785

In [162]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance_norm'].mean()

13.0885

In [163]:
dfQ_summary['describe_tree_edit_distance_norm'].mean()

12.442

In [164]:
dfQ_summary['describe_fewshots_tree_edit_distance_norm'].mean()

11.669

In [165]:
dfQ_summary['describe_tc_tree_edit_distance_norm'].mean()

13.208

In [166]:
dfQ_summary['describe_tc_fewshots_tree_edit_distance_norm'].mean()

11.9575

In [24]:
dfQ_summary['questions_syn_extrapolation_tree_edit_distance_norm'].mean()

12.6595

In [52]:
dfQ_summary['questions_syn_extrapolation_10_seed_data_tree_edit_distance_norm'].mean()

11.9355

In [92]:
dfQ_summary['questions_syn_extrapolation_50_seed_data_round_0_tree_edit_distance_norm'].mean()

11.9245